In [1]:
using AMLET, RDST, Distributions, ForwardDiff, GERALDINE

In [2]:
ngamma = 8

8

In [3]:
struct DummyBatch <: Batch
    IND::Array{LM_Individual}
    rng::MRG32k3a
end

import Base.iterate
function iterate(db::DummyBatch)
    state = 1
    reset_stream!(db.rng)
    if state <= length(db.IND)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end
function iterate(db::DummyBatch, state::Int = 1)
    if state <= length(db.IND)
        next_substream!(db.rng)
        return MLM_Individual(db.IND[state], db.rng, ngamma), state+1
    else
        return nothing
    end
end

iterate (generic function with 271 methods)

In [4]:
gen = MRG32k3aGen([125,8765423,4546,6434,645,76465])
betaGen = next_stream(gen)
monteCarlosGen = next_stream(gen)

Full state of MRG32k3a generator:
Cg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Bg = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]
Ig = [2459988903, 764778996, 4187205638, 4089700229, 517468743, 2169295582]

In [5]:
function beta(θ::Vector, γ::Vector)
    N = Normal.(θ[1:4], θ[5:8])
    return [quantile(N[i], γ[i]) for i in 1:4]
end

beta (generic function with 1 method)

In [6]:
gum = Gumbel()

Gumbel{Float64}(μ=0.0, θ=1.0)

In [7]:
function genIND(dists::Array{T, 1}, θ::Vector, m::Int) where T<: Distribution
    γ = rand(betaGen, 4)
    β = beta(θ, γ)
    N = length(dists)
    param = Array{Float64, 2}(undef, 0, N)
    for _ in 1:m
        param = [param ; (rand.(dists))']
    end
    best = argmax(param*β + rand(gum, m))
    return LM_Individual(param, best, 1)
end

genIND (generic function with 1 method)

In [8]:
θstar = [5,6,7,8, 1,1,1,1]

8-element Array{Int64,1}:
 5
 6
 7
 8
 1
 1
 1
 1

In [9]:
dists = [Normal(2, 4) for _ in 1:4]
genIND(dists, θstar, 5)

LM_Individual{Array{Float64,2}}([6.307410202783981 -0.5659116528732668 -0.4487674407830853 -2.9143008963605483; 6.883851259804331 -1.823243591344601 5.9309545470262535 -0.659519335896297; … ; 1.1323978825199141 5.3057602221031095 -2.508342779725468 -4.4709941489897185; 1.1555363614067804 4.092139061736238 -1.25875340179753 8.493009279423148], 5, 1)

In [10]:
db = DummyBatch([genIND(dists, θstar, 5) for _ in 1:5_000], monteCarlosGen);

In [11]:
mlm = MLM(db);

In [12]:
#linear utilities of mixed logit model where each parameters of beta are assumed to follow an 
#independant exponnential Distribution beta

function EU(θ::Vector, X::Matrix, γ::Vector)
    β = beta(θ, γ)
    return X*β
end

function EU_i(θ::Vector, X::Matrix, i::Int64, γ::Vector)
    β = beta(θ, γ)
    return X[i, :]'*β
end

function ∇EU(θ::Vector, X::Matrix, γ::Vector)
    return vcat([∇EU_i(θ, X, i, γ) for i in 1:size(X, 1)]...)
end

function ∇EU_i(θ::Vector, X::Matrix, i::Int64, γ::Vector)
    function tmp(x::Vector)
        return EU_i(x, X, i, γ)
    end
    return ForwardDiff.gradient(tmp, θ)
end

#uti = AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)


∇EU_i (generic function with 1 method)

In [13]:
#linear utilities of mixed logit model where each parameters of beta are assumed to follow an 
#independant exponnential Distribution beta

function EU(θ::Vector, X::Matrix, γ::Vector)
    β = θ[1:4] + θ[5:8] .* [quantile(Normal(0, 1), γ[i]) for i in 1:4]
    return X*β
end

function EU_i(θ::Vector, X::Matrix, i::Int64, γ::Vector)
    β = θ[1:4] + θ[5:8] .* [quantile(Normal(0, 1), γ[i]) for i in 1:4]
    return X[i, :]'*β
end

function ∇EU(θ::Vector, X::Matrix, γ::Vector)
    return vcat([∇EU_i(θ, X, i, rng) for i in 1:size(X, 1)]...)
end

function ∇EU_i(θ::Vector, X::Matrix, i::Int64, γ::Vector)
    function tmp(x::Vector)
        return EU_i(x, X, i, γ)
    end
    return ForwardDiff.gradient(tmp, θ)
end

uti = AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)


AMLET.LinearUtilitie(EU, ∇EU, EU_i, ∇EU_i)

In [14]:
complete_Model!(mlm, uti, 200)

(::AMLET.var"#∇F!#17"{DummyBatch,AMLET.LinearUtilitie,Int64}) (generic function with 2 methods)

In [15]:
mlm.f(ones(8))

0.6283046487348312

In [16]:
mlm.f(ones(8))

0.6283046487348312

In [17]:
grad1 = zeros(8)
grad2 = zeros(8)

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
mlm.∇f!(ones(8), grad1)
println(grad1)
mlm.∇f!(ones(8), grad2)
println(grad2)

[-0.03172395840609202, -0.08133359669893687, -0.1291392970797168, -0.1898400243166451, 0.10823340681370168, 0.10760000915975745, 0.102991839321354, 0.09142940803316829]
[-0.03172395840609202, -0.08133359669893687, -0.1291392970797168, -0.1898400243166451, 0.10823340681370168, 0.10760000915975745, 0.102991839321354, 0.09142940803316829]


In [19]:
grad1 == grad2

true

# Using the basic trust region (BTR) algorithm with BFGS approximation

In [20]:
using Pkg
Pkg.update("GERALDINE")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `git@github.com:JLChartrand/GERALDINE.jl.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


In [21]:
using GERALDINE

In [22]:
 x0 = 1.0*θstar

8-element Array{Float64,1}:
 5.0
 6.0
 7.0
 8.0
 1.0
 1.0
 1.0
 1.0

In [23]:
x = OPTIM_BFGS(mlm.f, mlm.∇f!, x0, nmax = 200, verbose = true)

[4.994820055414705, 6.00403279528557, 7.0011422060094155, 7.998797553212299, 0.998540836696756, 0.9981227612882464, 1.0031508414548884, 1.0012733238788152]
[4.989908275498458, 6.007804675461698, 7.002245753699512, 7.997679470690569, 0.9970191726828939, 0.996272100445839, 1.0062157609489064, 1.0024562236164043]
[4.887685946647102, 6.085234536809562, 7.025622216248882, 7.9748624706668325, 0.9623861715608777, 0.9563104565627158, 1.071641562103652, 1.0266249845518636]
[4.89654115289733, 6.077867351921331, 7.023908894589404, 7.977084149744145, 0.962823196122579, 0.958523874467713, 1.0675612033413047, 1.0244423185724028]
[4.90435855401779, 6.070008733819433, 7.022989231981862, 7.9795704105203304, 0.9580393696431334, 0.9580017358497134, 1.067104504701647, 1.0222981077285886]
[4.915695018926567, 6.054684685270495, 7.023330808489432, 7.984638436403262, 0.9362087354549116, 0.9501687241577093, 1.0757192328869127, 1.0187007011580242]
[4.917271141225082, 6.044246808880318, 7.026836728608741, 7.9883

([2.5865466176941534, 3.1955141908776827, 3.7069526591473396, 4.215254651223084, 0.4154774097276002, 0.4531362037542569, 0.5778536640132605, 0.5181965527061683], GERALDINE.BFGS([0.17681867745970653 -0.039986691869488555 … -0.004118115686766007 -0.011526955270287992; -0.039986691869488555 0.1734210517767294 … 0.003940713488435717 0.0071449679761193775; … ; -0.004118115686766007 0.003940713488435717 … 0.07796521306559537 0.021322173028795938; -0.011526955270287992 0.0071449679761193775 … 0.021322173028795938 0.06956545563620826], [1179.1711420952427 1459.0629194253231 … 315.02209838387034 301.1683989888428; 1459.0629194253045 1817.6895752352843 … 391.4888523929116 372.1954024036228; … ; 315.02209838381816 391.48885239285084 … 107.69349376345498 79.46815584363095; 301.16839898886707 372.19540240365876 … 79.4681558436546 97.49344252398], false), 54)

In [24]:
grad = zeros(8)

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:
mlm.∇f!(x[1], grad)

8-element Array{Float64,1}:
 -2.6759493466020002e-5
  2.8466532855661188e-5
 -2.0084439945731334e-5
  1.0462782798602907e-5
  1.471869719839486e-5 
 -2.638628023109127e-5 
  8.079021298790532e-6 
  1.5731966446007883e-5